In [224]:
import pandas as pd
import numpy as np
import connectteams as ct
import time
import openpyxl
%load_ext autoreload
%autoreload 2
import datetime

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [225]:
df1 = pd.read_excel(r"C:\Users\hussi\Downloads\Schedule-Export 2024-06-17 to 2024-06-23.xlsx")
df2 = pd.read_excel(r"C:\Users\hussi\Downloads\Schedule-Export 2024-06-24 to 2024-06-30.xlsx")
#merge df and df_1
df_two_week_schedule = pd.concat([df1, df2])
df_two_week_schedule.shape


(2296, 15)

In [226]:
# Convert Date column to datetime
df_two_week_schedule['Date'] = pd.to_datetime(df_two_week_schedule['Date'])
# Convert Start and End times to datetime.time
df_two_week_schedule['Start'] = pd.to_datetime(df_two_week_schedule['Start'], format='%I:%M%p').dt.time
df_two_week_schedule['End'] = pd.to_datetime(df_two_week_schedule['End'], format='%I:%M%p').dt.time


In [230]:
df_two_week_schedule
split_shifts = ct.split_shifts(df_two_week_schedule)
tag_holidays = ct.holiday_tagger(split_shifts)
extract_week_from_date = ct.get_info_from_date(tag_holidays)
time_sheet = ct.create_time_sheet(extract_week_from_date)



In [231]:
time_sheet

,Users,Month,Year,Holiday Hours,Regular Hours,Overtime Hours,Week
0,Aayanna Norman,6,2024,0.0,33.00,0.00,25
1,Aayanna Norman,6,2024,0.0,7.00,0.00,26
2,Abdel Munpen,6,2024,0.0,30.50,0.00,25
3,Abdel Munpen,6,2024,0.0,8.00,0.00,26
4,Abdul Khan,6,2024,0.0,33.78,0.00,25
...,...,...,...,...,...,...,...
585,Zachary Kirkendoll,6,2024,0.0,40.00,16.06,26
586,Zachary Kirkendoll,7,2024,0.0,2.00,0.00,27
587,Zariyana Canada,6,2024,0.0,40.00,3.00,25
588,Zariyana Canada,6,2024,0.0,35.00,0.00,26
